<a href="https://colab.research.google.com/github/mikeacquaviva/APS360-Leukaemia-Classification/blob/main/VAE_images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import numpy as np
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
from torchvision import datasets, transforms

In [ ]:
%%bash
git clone https://github.com/mikeacquaviva/APS360-Leukaemia-Classification.git

Cloning into 'APS360-Leukaemia-Classification'...
Updating files: 100% (7233/7233), done.


In [ ]:
preprocessed_data_dir = ("/content/gdrive/MyDrive/APS360/processed")
classes = ['benign', 'early', 'pre', 'pro']
data_transform = transforms.Compose([transforms.Resize((224, 224)),
                                      transforms.ToTensor()])

In [ ]:
data_dir = "/content/APS360-Leukaemia-Classification/images"
classes = ['benign', 'early', 'pre', 'pro']
data_transform = transforms.Compose([transforms.Resize((224, 224)),
                                      transforms.ToTensor()])
data = datasets.ImageFolder(data_dir, transform = data_transform)

#print(len(data))

In [ ]:
#code to retreive the data for input

In [ ]:
#from the tutorial
# dimensions of latent space
zdim = 25
featureDim = 224 * 224
# Variational Autoencoder
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()

        self.encConv1 = nn.Conv2d(3, 16, 5)
        self.encConv2 = nn.Conv2d(16, 32, 5)
        self.encFC1 = nn.Linear(featureDim, zdim)
        self.encFC2 = nn.Linear(featureDim, zdim)

        # Initializing the fully-connected layer and 2 convolutional layers for decoder
        self.decFC1 = nn.Linear(zdim, featureDim)
        self.decConv1 = nn.ConvTranspose2d(32, 16, 5)
        self.decConv2 = nn.ConvTranspose2d(16, 3, 5) #img channels is 3
        # encoder
        self.fc1 = nn.Linear(224 * 224, 400)
        self.relu = nn.ReLU()
        self.fc2m = nn.Linear(400, zdim)  # mu layer
        self.fc2s = nn.Linear(400, zdim)  # sd layer

        # decoder
        self.fc3 = nn.Linear(zdim, 400)
        self.fc4 = nn.Linear(400, 224 * 224)
        self.sigmoid = nn.Sigmoid()

    def encode(self, x):
        h1 = self.relu(self.fc1(x))
        return self.fc2m(h1), self.fc2s(h1)

    # reparameterize
    def reparameterize(self, mu, logvar):
        if self.training:
            std = logvar.mul(0.5).exp_()
            eps = std.data.new(std.size()).normal_()
            return eps.mul(std).add_(mu)
        else:
            return mu

    def decode(self, z):
        h3 = self.relu(self.fc3(z))
        return self.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, 224 * 224))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

In [ ]:
#from the tutorial
# loss function for VAE are unique and use Kullback-Leibler
# divergence measure to force distribution to match unit Gaussian
def loss_function(recon_x, x, mu, logvar):
    bce = F.binary_cross_entropy(recon_x, x.view(-1, 224 * 224))
    kld = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    kld /= batch_size * 224 * 224
    return bce + kld

In [ ]:
#from the tutorial
def train(model, num_epochs = 1, batch_size=10, learning_rate = 1e-3):
    model.train() #train mode so that we do reparameterization
    torch.manual_seed(42)
    
    train_loader = torch.utils.data.DataLoader(datasets.ImageFolder(data_dir, transform = data_transform),
               batch_size = batch_size, shuffle = True)
    
    optimizer = optim.Adam(model.parameters(), learning_rate)
    
    for epoch in range(num_epochs):
      for data in train_loader:  # load batch
          img, _ = data
          
          recon, mu, logvar = model(img)
          loss = loss_function(recon, img, mu, logvar) # calculate loss
          loss.backward()
          optimizer.step()
          optimizer.zero_grad()
      
      print('Epoch:{}, Loss:{:.4f}'.format(epoch+1, float(loss)))

In [ ]:
batch_size = 64

model = Autoencoder()
train(model, num_epochs = 100, batch_size = batch_size)

In [ ]:
# generate random samples in latent space
model.eval()
sample = torch.randn(64, zdim)
sample = model.decode(sample)

import matplotlib.pyplot as plt
imgs = sample.data.view(64, 224, 224).numpy()
plt.imshow(imgs[4])

In [ ]:
# display images
for k in range(1):
    plt.figure(figsize=(8, 8))

    for i, item in enumerate(imgs):
        plt.subplot(8, 8, i+1)
        plt.imshow(item)

In [ ]:
#new autoencoder test
class Flatten(nn.Module):
  def forward(self, input):
    return input.view(input.size(0), -1)

class ConvAutoencoder(nn.Module):
    def __init__(self):
        super(ConvAutoencoder, self).__init__()
        self.encoder = nn.Sequential( # like the Composition layer you built
            nn.Conv2d(3, 16, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 32, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, 7),
            Flatten()
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(64, 32, 7),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 16, 3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 1, 3, stride=2, padding=1, output_padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [ ]:
def trainConv(model, num_epochs=5, batch_size=64, learning_rate=1e-3):
    torch.manual_seed(42)
    criterion = nn.MSELoss() # mean square error loss
    optimizer = torch.optim.Adam(model.parameters(),
                                 lr=learning_rate, 
                                 weight_decay=1e-5)
    train_loader = torch.utils.data.DataLoader(mnist_data, 
                                               batch_size=batch_size, 
                                               shuffle=True)
    outputs = []
    for epoch in range(num_epochs):
        for data in train_loader:
            img, _ = data
            recon = model(img)
            loss = criterion(recon, img)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

        print('Epoch:{}, Loss:{:.4f}'.format(epoch+1, float(loss)))
        outputs.append((epoch, img, recon),)
    return outputs

In [ ]:
modelConv = ConvAutoencoder()
max_epochs = 20
outputs1 = train(model, num_epochs=max_epochs)